In [36]:
import streamlit as st
import json
import plotly.express as px
import pandas as pd
import geopandas as gpd
import geojson
import folium
import numpy as np
import datetime
from shapely.geometry import Polygon, mapping

dataset = 'TILL6022_Emission_Dataset.csv'
df = pd.read_csv(dataset, delimiter=',', encoding='ISO-8859-1') #,parse_dates = ['date']
country_bb = pd.read_csv('country_bb.csv', delimiter=',', encoding='ISO-8859-1')
# country_bb.set_index('country', inplace=True)
#df.set_index('country', inplace=True)
#sf = df[df.date == '2019-01-01']
# vf = df[df.sector == 'Total']
df.date = pd.to_datetime(df.date)
df_filt = df.groupby(['country','sector']).resample('MS',on='date').sum()    #D for Day, MS for month, QS to quarter, Y for year
df_filt = df_filt.reset_index()
df_filt.date = np.datetime_as_string(df_filt.date, unit='D')
df_filt.dtypes


country     object
sector      object
date        object
co2        float64
dtype: object

Useful commands for geopandas
#ar = gdf[gdf.ADMIN == 'Aruba'] #To find the country Aruba
#print(ar.area)                 #Print area of country Aruba
# gdf.ADMIN['Aruba']
#counties['features'][3]['geometry'] access coordinates and type

In [2]:
# with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries.geojson") as f:
#     counties = geojson.load(f)
# gdf = gpd.GeoDataFrame.from_features(counties["features"], crs=4326)
# mask = gdf.area >2    # Function to remove smaller countries to speed up 
# gdf_selec = gdf.loc[mask]
# gdf_selec.head()
# ar = gdf[gdf.ADMIN == 'China']['geometry'] # access geometry data of China

#counties['features'][3]['geometry'] access coordinates and type
#counties['features']['properties']
with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries_simp.geojson") as f:
    counties = json.load(f)
with open("B:\Documenten\TU_Delft\Master\Module_1\Python\Assignment\TIL6022\countries_simp.geojson") as f:
    counties = json.load(f)
# gdf = gpd.GeoDataFrame.from_features(counties["features"], crs=4326)


In [3]:
# dept_geo = [0]*len(counties['features'])

# Get data from dataset (using json.loads to transform string into dict for geoshape column)
# cd_depts = gdf.ADMIN.tolist()
# dept_geo[0] = counties['features'][0]['geometry']['coordinates']
counties['features'][0]

{'type': 'Feature',
 'geometry': None,
 'properties': {'ADMIN': 'Aruba', 'ISO_A3': 'ABW'}}

In [40]:
shapes = [0]*len(counties['features'])
for feature in range(len(counties['features'])):
  shapes[feature] = counties['features'][feature]['geometry']['coordinates']
  # for cood in range(len(feature['geometry']['coordinates'])):
    # shapes[cood] = feature['geometry'][cood]['coordinates']
cd_depts = gdf.ADMIN.tolist()
for cd_dept, shape in zip(cd_depts, shapes):
    
    # Mapping information from data to GeoJSON file
    dept_geo.append({
        'type': 'Feature',
        'geometry': shape,
        'id':cd_dept
    })

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo_ok = {'type': 'FeatureCollection', 'features': dept_geo}

In [24]:
# import pandas as pd
# import plotly
# import numpy as np


# from urllib.request import urlopen
# import json
# with urlopen('https://opendata.arcgis.com/datasets/c572ffec290b42768f80e43d31ac53a6_0.geojson') as response:
#     counties = json.load(response)
    
# df = pd.read_excel('testdata.xlsx',dtype={"cty19cd": str})
# df.columns=['CODE','NAME','VALUE']

# import plotly.express as px

# fig = px.choropleth(df, geojson=counties, locations='CODE', featureidkey="properties.cty19cd", color='VALUE',
#                            color_continuous_scale="Inferno",
#                             scope="europe")

# fig.update_geos( fitbounds="locations", visible=True)
# fig.write_html('maptest.html',auto_open=True)

In [25]:
# map = folium.Map(
#     location=[
#         gdf_selec.geometry.bounds.miny.mean(),
#         gdf_selec.geometry.bounds.minx.mean()
#     ],
#     zoom_start=1
# )

# gjson = folium.features.GeoJson(
#     gdf_selec,
# ).add_to(map)
# map

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo = []

# Get data from dataset (using json.loads to transform string into dict for geoshape column)
cd_depts = gdf.ADMIN.tolist()
shapes = gdf.geometry
cd = mapping(shapes)
# # Iterate over lines
for cd_dept, shape in zip(cd_depts, shapes):
    
    # Mapping information from data to GeoJSON file
    dept_geo.append({
        'type': 'Feature',
        'geometry': shape,
        'id':cd_dept
    })

# Encapsulate dept_geo in a real GeoJSON formatted file
dept_geo_ok = {'type': 'FeatureCollection', 'features': dept_geo}


NameError: name 'gdf' is not defined

In [62]:
# fig = px.choropleth(vf, geojson=gdf_selec, color="co2",
#                     locations="ISO-A3", featureidkey="properties.district",
#                     projection="mercator"
#                    )
# fig.update_geos(fitbounds="locations", visible=False)
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
given_time = st.selectbox('Select time period to visualise', ['Day','Month','Quartile','Year'])
if given_time == 'Day':
    date_filt = 'D'
elif given_time == 'Month':
    date_filt = 'MS'
elif given_time == 'Quartile':
    date_filt = 'QS'
elif given_time == 'Year':
    date_filt = 'Y'

given_sector = st.selectbox('Select sector to visualise', df.sector.unique())

default_country = int(np.where(df['country'].unique() == 'WORLD')[0])
given_country = st.selectbox('Select country to visualise', df.country.unique(), index = default_country)
# given_country = "China"
# country_bb.country.index(given_country)
# country_bb.country[(country_bb.country.values==given_country).any(0)]
if (given_country == 'WORLD' or given_country == 'ROW' or given_country == 'EU27 & UK'):
    latrange = [-90,90]
    lonrange = [-180,180]
elif given_country == 'US':
    given_country == 'United States'
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin']),float(given_country_bb['latmax'])]
    lonrange = [float(given_country_bb['lonmin']),float(given_country_bb['lonmax'])]
else:
    given_country_bb = country_bb[country_bb.country == given_country]
    latrange = [float(given_country_bb['latmin']),float(given_country_bb['latmax'])]
    lonrange = [float(given_country_bb['lonmin']),float(given_country_bb['lonmax'])]


In [63]:
fig = px.choropleth(df_filt[df_filt.sector =='Domestic Aviation'], 
                    # geojson= counties,
                    # featureidkey='ISO_A3',
                    locations= 'country',
                    locationmode= "country names",
                    color='co2',
                    #hover_name="country",
                    animation_frame= 'date'
                    #zoom=2,
)
fig.update_geos(lataxis_range=latrange, lonaxis_range=lonrange)  
fig.update_layout(
    title_text = "CO2 emmissions transport sector",
    geo = dict(
        showframe = False,
        showcoastlines = True,
        projection_type = "equirectangular",
    ),
    
                    #color_continuous_scale="Blues_r
    height=1000,
    width  = 1080
    #margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()